# Renforcement

In [ ]:
### Import Lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import locale
import dateparser
import torch
from torch import tensor
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import os
from tqdm.auto import tqdm

In [ ]:
df = pd.read_csv("../data/avis/df_cleaned.csv",index_col = 0)
df.head()

In [ ]:
df.info()

### Remettre en datetime

In [ ]:
### Retirer les minutes / secondes / décalage
df["date_avis"] = df["date_avis"].str.split(" ").str[0]
df["date_experience"] = df["date_experience"].str.split(" ").str[0]

### Transformation en datetime
df["date_avis"] = pd.to_datetime(df["date_avis"], format='%Y-%m-%d')
df["date_experience"] = pd.to_datetime(df["date_experience"], format='%Y-%m-%d')

df.info()

### Ajouter les colonnes jour / mois / heure / année et les catégoriser + rajouter longueur des textes

In [ ]:
### Ajout des colonnes jour / mois / heure / année pour avis
df["jour_avis"] = df["date_avis"].dt.day
df["mois_avis"] = df["date_avis"].dt.month
df["heure_avis"] = df["date_avis"].dt.hour
df["annee_avis"] = df["date_avis"].dt.year

df["jour_experience"] = df["date_experience"].dt.day
df["mois_experience"] = df["date_experience"].dt.month
df["heure_experience"] = df["date_experience"].dt.hour
df["annee_experience"] = df["date_experience"].dt.year

### Catégoriser en jour et nuit
df["periode_avis"] = df["heure_avis"].apply(lambda x: "jour" if 6 <= x <= 22 else "nuit")
df["longueur_texte"] = df["text_total"].apply(len)

df.head()

In [ ]:
# removing stopwords
import spacy
nlp = spacy.load("fr_core_news_lg")
df['text_total'] = df['text_total'].fillna('').astype(str)
df['text_total'] = [message.lower() for message in df['text_total']]
stop_words = spacy.lang.fr.stop_words.STOP_WORDS

def stop_stop(review):
    filtre = []
    texte = nlp(review)
    liste_texte = list(texte)
    for word in liste_texte:
        if str(word) not in stop_words:
            filtre.append(str(word))
    return(" ".join(filtre))

liste_filtrée =[]
round_count = 1
for review in df.text_total:
    if round_count % 100 == 0:
        print(f"{round_count} reviews passed of {len(df)} - {round(round_count/len(df)*100,2)}% done")
    filt_review = stop_stop(review)
    liste_filtrée.append(filt_review)
    round_count = round_count + 1
df["text_stop"] = liste_filtrée

In [ ]:
# Extracting lemmas
batch_size = 1000
liste_lemma = []
i = 0
for doc in nlp.pipe(df.text_stop, batch_size=batch_size):
    i +=1
    lemmas = [token.lemma_ for token in doc]
    liste_lemma.append(" ".join(lemmas))
    print(i ,"reviews passed")
df['text_lemma'] = liste_lemma

saving the consolidated dataset

In [ ]:
df.to_csv("../data/avis/df_clean_noYC_lemma.csv")